<a href="https://colab.research.google.com/github/ENuel20/Joyce/blob/master/AIagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langgraph
!pip install langchain-community
!pip install langchain-groq
!pip install langchain-anthropic
!pip install langchain-openai

!pip install langchain-nomic[local]
!pip install tiktoken
!pip install langchainhub
!pip install chromadb
!pip install langchain-text-splitters
!pip install bs4
!pip install python-dotenv
!pip install sentence_transformers

  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.32.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached posthog-3.7.4-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.20.1-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.28.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.49b2-py3-none-any.whl.metadata (2.1 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 8.1 MB/s et

In [ ]:

import os
from langchain_groq import ChatGroq

# Set your Groq API key as an environment variable
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual key

# Instantiate ChatGroq with your desired model
chat_model = ChatGroq(model="mixtral-8x7b-32768")  # Replace with your Groq model name

# Invoke the model
response = chat_model.invoke("Who was the first man on the moon?")
print(response)

content='The first man to walk on the moon was Neil Armstrong on July 20, 1969, as part of the Apollo 11 mission. Armstrong\'s famous words, "That\'s one small step for man, one giant leap for mankind," were spoken as he stepped off the lunar module Eagle onto the moon\'s surface.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 17, 'total_tokens': 97, 'completion_time': 0.127033495, 'prompt_time': 0.003111571, 'queue_time': 0.011904399, 'total_time': 0.130145066}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None} id='run-c444c728-d27d-45ce-b491-6fa7ed5927fb-0' usage_metadata={'input_tokens': 17, 'output_tokens': 80, 'total_tokens': 97}


response = chat_model.invoke("why rocket boosters")
response.content

In [ ]:
response.usage_metadata


{'input_tokens': 13, 'output_tokens': 346, 'total_tokens': 359}

In [ ]:
response.dict

<bound method BaseModel.dict of AIMessage(content='Rocket boosters, also known as solid rocket boosters or liquid rocket boosters, are used to provide additional thrust to a launch vehicle during the initial phase of flight. There are several reasons why rocket boosters are used:\n\n1. Increased thrust: Rocket boosters can provide a significant amount of additional thrust, which helps the launch vehicle to overcome the force of gravity and accelerate to higher speeds more quickly.\n2. Shorter launch duration: By providing a burst of extra thrust at the beginning of the flight, rocket boosters can help to reduce the overall duration of the launch, which can be important for missions with tight timelines or for reducing the amount of time that the launch vehicle is exposed to potential hazards.\n3. Improved payload capacity: Rocket boosters can help to increase the payload capacity of a launch vehicle by providing the additional thrust needed to lift heavier payloads into orbit.\n4. Flexibility: Rocket boosters can be used in a variety of configurations, depending on the specific needs of the mission. For example, some launch vehicles use multiple, smaller boosters, while others use a single, larger booster.\n5. Reusability: Some rocket boosters are designed to be reusable, which can help to reduce the cost of launching missions into space. For example, the Space Shuttle used solid rocket boosters that were recovered and refurbished after each flight.\n\nOverall, rocket boosters are an important tool for launching spacecraft into orbit, and they provide a number of benefits that can help to improve the performance and efficiency of launch vehicles.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 346, 'prompt_tokens': 13, 'total_tokens': 359, 'completion_time': 0.553599804, 'prompt_time': 0.002429673, 'queue_time': 0.010848767, 'total_time': 0.556029477}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-a0c25de3-f26d-4b7e-ac63-4ae4554440d2-0', usage_metadata={'input_tokens': 13, 'output_tokens': 346, 'total_tokens': 359})>

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages




class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


llm = ChatGroq(model="mixtral-8x7b-32768")


def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()


In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    user_input = input("user: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("goodbye!")
        break
    elif user_input == "":  # Check for empty input
        user_input = "what do you know about langraph?"
        print("User:", user_input)

    stream_graph_updates(user_input)

user: fish
Assistant: Sure, I can help you with information about fish! Fish are a group of aquatic organisms that live in water. They are characterized by their gills, which they use to extract oxygen from the water, and by their fins, which they use to propel themselves through the water. Fish come in a wide variety of shapes, sizes, and colors, and they can be found in both freshwater and saltwater environments all over the world.

There are many different species of fish, and they play important roles in ecosystems as both predators and prey. Some fish are also important sources of food for humans, and fishing is a major industry in many parts of the world.

Fish have been an important part of human culture and cuisine for thousands of years. They are often featured in art, literature, and mythology, and they are an important part of many religious and spiritual traditions.

Do you have any specific questions about fish that you would like me to answer? I'm here to help!
user: 
Use

In [ ]:
import getpass
import os

def _set_env(var: str):
  if not os.environ.get(var):
    os.environ[var] = getpass.getpass(f"Enter your {var}: ")
_set_env("TAVILY_API_KEY")


Enter your TAVILY_API_KEY: ··········


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(max_results=4)
tools = [search_tool]
search_tool.invoke("who is the current Administrator of the National Aeronautics and Space Administration (NASA)?")


[{'url': 'https://www.nasa.gov/nasa-leadership/',
  'content': "The agency's current Administrator, Deputy Administrator, and Associate Administrator. ... Bill Nelson, Administrator. Sen. Bill Nelson was sworn in as the 14th NASA Administrator on May 3, 2021, tasked with carrying out the Biden-Harris administration's vision for the agency. ... National Aeronautics and Space Administration. NASA explores"},
 {'url': 'https://spaceexplored.com/2024/12/04/jared-isaacman-as-nasa-administrator-does-he-have-the-right-stuff/',
  'content': "As NASA Administrator, Isaacman will have to show that he will continue NASA's goal of being a non-biased facilitator of public-private partnerships. ... The current outlook of NASA under Administrator Bill Nelson is that Artemis will eventually work and get us back to the Moon, ... NASA (National Aeronautics and Space Administration) Donald"},
 {'url': 'https://www.nasa.gov/people/nasa-administrator-bill-nelson/',
  'content': "Senator Bill Nelson was swo